In [10]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from pyclustering.cluster.kmedoids import kmedoids
from pyclustering.cluster import cluster_visualizer
from pyclustering.utils import calculate_distance_matrix
from sklearn.metrics import silhouette_score

In [ ]:
dataset_path = '/home/awesomebaba/Desktop/python/sih/ml_in_sih/recommendation/new.csv'  # Replace with your dataset path
df = pd.read_csv(dataset_path)

In [12]:

features = ['latitude','longitude','height']

In [13]:
X = df[features].values

In [14]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:

df_sample = df.sample(300, random_state=42)
X_sample = df_sample[features].values
X_sample_scaled = scaler.transform(X_sample)

In [16]:
#corner_coords = [
    #(23.054133, 72.516336),  # Corner 1
    #(23.054075, 72.523878),  # Corner 2
    #(23.030544, 72.532173),  # Corner 3
    #(23.029959, 72.510126)   # Corner 4

lat_min, lat_max = df_sample['Latitude'].min(), df_sample['Latitude'].max()
lon_min, lon_max = df_sample['Longitude'].min(), df_sample['Longitude'].max()

In [17]:
#lat_min = min(corner[0] for corner in corner_coords)
#lat_max = max(corner[0] for corner in corner_coords)
#lon_min = min(corner[1] for corner in corner_coords)
#lon_max = max(corner[1] for corner in corner_coords)

In [ ]:

lat_min, lat_max = df_sample['latitude'].min(), df_sample['latitude'].max()
lon_min, lon_max = df_sample['longitude'].min(), df_sample['l0ongitude'].max()

In [19]:
lat_intervals = np.linspace(lat_min, lat_max, 4)  # Divide latitude range into 3 intervals
lon_intervals = np.linspace(lon_min, lon_max, 4)  # Divide longitude range into 3 intervals

In [20]:
medoid_coords = [(lat, lon) for lat in lat_intervals for lon in lon_intervals]
medoid_indices = []

In [21]:
for lat, lon in medoid_coords:
    distances = np.sqrt((df['latitude'] - lat)**2 + (df['longitude'] - lon)**2)
    closest_point_idx = distances.idxmin()
    medoid_indices.append(closest_point_idx)

In [ ]:
medoid_indices = list(set(medoid_indices))[:6]  # Select up to 10 medoids

In [23]:
distance_matrix = calculate_distance_matrix(X_scaled)

In [ ]:
kmedoids_instance = kmedoids(distance_matrix, medoid_indices) # Enable logging to track progress
kmedoids_instance.process()

In [ ]:
clusters = kmedoids_instance.get_clusters()
medoids = kmedoids_instance.get_medoids()

In [ ]:
labels = np.zeros(len(X_scaled))
for cluster_id, cluster_points in enumerate(clusters):
    for point in cluster_points:
        labels[point] = cluster_id

In [ ]:
silhouette_avg = silhouette_score(X_sample_scaled, labels)
print(f"Silhouette Score: {silhouette_avg}")


In [ ]:

df_sample['Cluster'] = labels.astype(int)
df_sample.to_csv('sample_clustered_dataset.csv', index=False)
print("Clustered sample dataset saved to 'sample_clustered_dataset.csv'")